In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler #sirve para estandarizar los datos
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import RepeatedStratifiedKFold #hace CrossValidation con grid
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/refs/heads/main/winequality-red.csv", sep=';')
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [ ]:
df['aceptable']  = np.where(df['quality']>=6,1,0)

In [ ]:
df['aceptable'].value_counts()

,count
aceptable,
1,855
0,744


In [ ]:
df['quality'].unique()

array([5, 6, 7, 4, 8, 3])

In [ ]:
target = 'quality'
X,y =  df.drop(columns=[target , 'aceptable']) , df['aceptable'] # change target to normal

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,
                                                    stratify=y, random_state=123)
#stratify asegura que la distribución de clases en la variable objetivo (y) sea la misma
#tanto en train como en test

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
#ROBOT Y RESULTADO SIN GRID:
robot = KNeighborsClassifier(n_neighbors=10)
robot.fit(X_train_scaled, y_train)
preds_scaled = robot.predict(X_test_scaled)
print(classification_report(y_test, preds_scaled))


              precision    recall  f1-score   support

           0       0.71      0.74      0.73       149
           1       0.77      0.73      0.75       171

    accuracy                           0.74       320
   macro avg       0.74      0.74      0.74       320
weighted avg       0.74      0.74      0.74       320



In [ ]:
#ARQUITECTURA DEL GRID USANDO KNN

def knn(X_train, y_train):
    model = KNeighborsClassifier()
    n_neighbors=[3,4,5,6,7,8,9, 10, 13, 21, 23, 24, 25, 30]
    weights=['uniform', 'distance']
    penalty = ['l2','l1'] #penalizaciones basadas por la regularizacion
    grid = dict(n_neighbors=n_neighbors, weights=weights)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv,
                           scoring='accuracy',error_score='raise')
    grid_result = grid_search.fit(X_train, y_train)
    return  grid_result.best_estimator_

In [ ]:
model_knn_scaled = knn(X_train_scaled, y_train)
y_pred_scaled = model_knn_scaled.predict(X_test_scaled)
print(classification_report(y_test, y_pred_scaled))
#recall = 0 ESPECIFICIDAD
#recall = 1 SENSIBILIDAD

              precision    recall  f1-score   support

           0       0.82      0.77      0.79       149
           1       0.81      0.85      0.83       171

    accuracy                           0.81       320
   macro avg       0.81      0.81      0.81       320
weighted avg       0.81      0.81      0.81       320



In [ ]:
model_knn_scaled

KNeighborsClassifier(n_neighbors=30, weights='distance')

Conclusión:

El modelo optimizado (model_knn) tiene mejores métricas de rendimiento, en comparación con el modelo inicial. La optimización usando GridSearchCV fue mejor, generando una precision del 81%.